In [98]:
import math
import random

In [110]:
def defensive_heuristic1(color, wl, bl):
    if color is 1:
        return 2*len(wl) + random.random()
    else:
        return 2*len(bl) + random.random()


In [111]:
def offensive_heuristic1(color, wl, bl):
    if color is 1:
        return 2*(30-len(bl)) + random.random()
    else:
        return 2*(30-len(wl)) + random.random()


In [101]:
def init_pieces():
    wl = set()
    bl = set()
    for i in range(8):
        bl.add((0, i))
        bl.add((1, i))
        wl.add((6, i))
        wl.add((7, i))
    return bl, wl

In [102]:
bl, wl = init_pieces()
#print(bl)

In [103]:
# returns boolean, and the piece eaten
def viable(color, new_pos, straight, bl, wl):
    if color == 1:  # black moving
        if new_pos[0] > 0 and new_pos[1] > 0 and new_pos[0] < 8 and new_pos[1] < 8:
            # check whether moving onto black piece
            if new_pos in bl:
                return False, False
            
            # check whether moving onto white piece
            if straight:
                if new_pos in wl:
                    return False, False
            else: # may eat some piece
                if new_pos in wl:
                    return True, True, new_pos
                
            return True, False
        
        else:
            return False, False
    
    else:           # white moving
        if new_pos[0] > 0 and new_pos[1] > 0 and new_pos[0] < 8 and new_pos[1] < 8:
            # check whether moving onto white piece
            if new_pos in wl:
                return False, False
            
            # check whether moving onto black piece
            if straight:
                if new_pos in bl:
                    return False, False
            else: # may eat some piece
                if new_pos in bl:
                    return True, True, new_pos
                
            return True, False
        
        else:
            return False, False

In [114]:
def base_case(bl, wl):
    if (not wl) or (not bl):
        #print(wl)
        #print(bl)
        return True
    for p in bl:
        if p[0] == 7:
            return True
    for p in wl:
        if p[0] == 0:
            return True
    
    return False

In [129]:
# black --> 1
# white --> -1
node_expanded = 0

def maxmove(color, bl, wl, depth, init_color):
    global node_expanded
    node_expanded += 1
    # base case: all pieces of one agent die   or   one pieces of a agent reach the other side
    if base_case(bl, wl) or depth == 0:
        return offensive_heuristic1(init_color, bl, wl), -1    # the second thing return is a dummy 
    
    if color == 1:
        cur_max = -math.inf
        best_action = (-1, bl, wl)
        # also record best action
        for p in bl:
            
            # left forward
            new_pos = (p[0]+1, p[1]+1)
            via = viable(1, new_pos, False, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                if via[1]:
                    new_wl.remove(via[2])
                    
                val = minmove(-1, new_bl, new_wl, depth-1, init_color)
                if (val[0] > cur_max):
                    cur_max = val[0]
                    best_action = (-1, new_bl, new_wl)
                    
            # forward
            new_pos = (p[0]+1, p[1])
            via = viable(1, new_pos, True, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                val = minmove(-1, new_bl, new_wl, depth-1, init_color)
                if (val[0] > cur_max):
                    cur_max = val[0]
                    best_action = (-1, new_bl, new_wl)
                
            # right forward
            new_pos = (p[0]+1, p[1]-1)        
            via = viable(1, new_pos, False, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                if via[1]:
                    new_wl.remove(via[2])
                    
                val = minmove(-1, new_bl, new_wl, depth-1, init_color)
                if (val[0] > cur_max):
                    cur_max = val[0]
                    best_action = (-1, new_bl, new_wl)
                    
           
        # return the most promising result after loop through all pieces
        return cur_max, best_action
    else:  # color = -1
        cur_max = -math.inf
        best_action = (1, bl, wl)
        # also record best action
        for p in wl:
            
            # left forward
            new_pos = (p[0]-1, p[1]-1)
            via = viable(-1, new_pos, False, bl, wl)
            if via[0]:
                new_wl = wl.copy()
                new_wl.remove(p)
                new_wl.add(new_pos)
                new_bl = bl.copy()
                if via[1]:
                    new_bl.remove(via[2])
                    
                val = minmove(1, new_bl, new_wl, depth-1, init_color)
                if (val[0] > cur_max):
                    cur_max = val[0]
                    best_action = (1, new_bl, new_wl)
                    
            # forward
            new_pos = (p[0]-1, p[1])
            via = viable(-1, new_pos, True, bl, wl)
            if via[0]:
                new_wl = wl.copy()
                new_wl.remove(p)
                new_wl.add(new_pos)
                new_bl = bl.copy()
                val = minmove(1, new_bl, new_wl, depth-1, init_color)
                if (val[0] > cur_max):
                    cur_max = val[0]
                    best_action = (1, new_bl, new_wl)
                
            # right forward
            new_pos = (p[0]-1, p[1]+1)        
            via = viable(-1, new_pos, False, bl, wl)
            if via[0]:
                new_wl = wl.copy()
                new_wl.remove(p)
                new_wl.add(new_pos)
                new_bl = bl.copy()
                if via[1]:
                    new_bl.remove(via[2])
                    
                val = minmove(1, new_bl, new_wl, depth-1, init_color)
                if (val[0] > cur_max):
                    cur_max = val[0]
                    best_action = (1, new_bl, new_wl)
                    
           
        # return the most promising result after loop through all pieces
        return cur_max, best_action

In [130]:

if True:
    best = 5
print(best)

5


In [131]:
# black --> 1
# white --> -1

def minmove(color, bl, wl, depth, init_color):
    global node_expanded
    node_expanded += 1
    # base case: all pieces of one agent die   or   one pieces of a agent reach the other side
    if base_case(bl, wl) or depth == 0:
        return offensive_heuristic1(init_color, bl, wl), -1   # the second thing return is a dummy 
    
    if color == 1:
        cur_min = math.inf
        best_action = (-1, bl, wl)
        # also record best action
        for p in bl:
            
            # left forward
            new_pos = (p[0]+1, p[1]+1)
            via = viable(1, new_pos, False, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                if via[1]:
                    new_wl.remove(via[2])
                    
                val = maxmove(-1, new_bl, new_wl, depth-1, init_color)
                if (val[0] < cur_min):
                    cur_min = val[0]
                    best_action = (-1, new_bl, new_wl)
                    
            # forward
            new_pos = (p[0]+1, p[1])
            via = viable(1, new_pos, True, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                val = maxmove(-1, new_bl, new_wl, depth-1, init_color)
                if (val[0] < cur_min):
                    cur_min = val[0]
                    best_action = (-1, new_bl, new_wl)
                
            # right forward
            new_pos = (p[0]+1, p[1]-1)        
            via = viable(1, new_pos, False, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                if via[1]:
                    new_wl.remove(via[2])
                    
                val = maxmove(-1, new_bl, new_wl, depth-1, init_color)
                if (val[0] < cur_min):
                    cur_min = val[0]
                    best_action = (-1, new_bl, new_wl)
                    
           
        # return the most promising result after loop through all pieces
        return cur_min, best_action
    else:  # color = -1
        cur_min = math.inf
        best_action = (-1, bl, wl)
        # also record best action
        for p in wl:
            
            # left forward
            new_pos = (p[0]-1, p[1]-1)
            via = viable(-1, new_pos, False, bl, wl)
            if via[0]:
                new_wl = wl.copy()
                new_wl.remove(p)
                new_wl.add(new_pos)
                new_bl = bl.copy()
                if via[1]:
                    new_bl.remove(via[2])
                    
                val = maxmove(1, new_bl, new_wl, depth-1, init_color)
                if (val[0] < cur_min):
                    cur_min = val[0]
                    best_action = (1, new_bl, new_wl)
                    
            # forward
            new_pos = (p[0]-1, p[1])
            via = viable(-1, new_pos, True, bl, wl)
            if via[0]:
                new_wl = wl.copy()
                new_wl.remove(p)
                new_wl.add(new_pos)
                new_bl = bl.copy()
                val = maxmove(1, new_bl, new_wl, depth-1, init_color)
                if (val[0] < cur_min):
                    cur_min = val[0]
                    best_action = (1, new_bl, new_wl)
                
            # right forward
            new_pos = (p[0]-1, p[1]+1)        
            via = viable(-1, new_pos, False, bl, wl)
            if via[0]:
                new_wl = wl.copy()
                new_wl.remove(p)
                new_wl.add(new_pos)
                new_bl = bl.copy()
                if via[1]:
                    new_bl.remove(via[2])
                    
                val = maxmove(1, new_bl, new_wl, depth-1, init_color)
                if (val[0] < cur_min):
                    cur_min = val[0]
                    best_action = (1, new_bl, new_wl)
                    
           
        # return the most promising result after loop through all pieces
        return cur_min, best_action

In [136]:
bl, wl = init_pieces()
color = -1
counter = 0
while not base_case(bl, wl):
    _, bl, wl = maxmove(color, bl, wl, 3, color)[1]
    print(counter)
    print("white list")
    print(sorted(wl))
    print("black list")
    print(sorted(bl))
    color = -color
    counter += 1
    
if (not wl):
    print("Black wins")
elif (not bl):
    print("White wins")
else:    
    for p in bl:
        if p[0] == 7:
            print("Black wins")
    for p in wl:
        if p[0] == 0:
            print("White wins")
            
print(node_expanded)
print(node_expanded / counter)
    #print(wl)
    #print(bl)

0
white list
[(5, 2), (6, 0), (6, 1), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)]
black list
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7)]
1
white list
[(5, 2), (6, 0), (6, 1), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)]
black list
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 4), (1, 5), (1, 6), (1, 7), (2, 3)]
2
white list
[(5, 2), (5, 4), (6, 0), (6, 1), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)]
black list
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 4), (1, 5), (1, 6), (1, 7), (2, 3)]
3
white list
[(5, 2), (5, 4), (6, 0), (6, 1), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)]
black list
[

37
white list
[(5, 7), (6, 0), (6, 1), (7, 0), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)]
black list
[(0, 0), (0, 3), (0, 4), (0, 6), (1, 1), (1, 2), (1, 3), (1, 4), (2, 6), (4, 4)]
38
white list
[(5, 7), (6, 0), (6, 1), (6, 3), (7, 0), (7, 2), (7, 4), (7, 5), (7, 6), (7, 7)]
black list
[(0, 0), (0, 3), (0, 4), (0, 6), (1, 1), (1, 2), (1, 3), (1, 4), (2, 6), (4, 4)]
39
white list
[(5, 7), (6, 0), (6, 1), (6, 3), (7, 0), (7, 2), (7, 4), (7, 5), (7, 6), (7, 7)]
black list
[(0, 0), (0, 3), (0, 4), (0, 6), (1, 1), (1, 3), (1, 4), (2, 3), (2, 6), (4, 4)]
40
white list
[(5, 3), (5, 7), (6, 0), (6, 1), (7, 0), (7, 2), (7, 4), (7, 5), (7, 6), (7, 7)]
black list
[(0, 0), (0, 3), (0, 4), (0, 6), (1, 1), (1, 3), (1, 4), (2, 3), (2, 6), (4, 4)]
41
white list
[(5, 7), (6, 0), (6, 1), (7, 0), (7, 2), (7, 4), (7, 5), (7, 6), (7, 7)]
black list
[(0, 0), (0, 3), (0, 4), (0, 6), (1, 1), (1, 3), (1, 4), (2, 3), (2, 6), (5, 3)]
42
white list
[(5, 7), (6, 0), (6, 1), (6, 2), (7, 0), (7, 4), (7, 5), (7